# A Developer's Introduction to `schedview`

## Notebook setup

### Notebook formatting

`lab_black` is nice for notebook development, but remove or comment it out when running the notebook in Times Square.

In [ ]:
%load_ext lab_black

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

### Development version of modules

At this stage of the porject, the versions of `schedview` and the scheduler-related modules in the defaul `LSST` envionment at the USDF are usually significantly out of date.

So, to get the latest versions, check out the versions you want, build them, and modify the path to point to them. For example, if your development directories are in `/sdf/data/rubin/user/${USER}/devel`, then you can set the modules you want to load devel versions of, then add the relevant entries to the path:

In [ ]:
# To change which devel modules to use, just comment or uncomment relevant elements in this list:
devel_module_names = [
    #    'uranography',
    "rubin_scheduler",
    "rubin_sim",
    "schedview",
]

import os
import sys
import pwd
from pathlib import Path

username = pwd.getpwuid(os.getuid())[0]
devel_module_path = Path("/sdf/data/rubin/user").joinpath(username, "devel")
for module_name in devel_module_names:
    sys.path.insert(0, devel_module_path.joinpath(module_name).as_posix())

### Import necessary modules

In [ ]:
import pandas as pd
import warnings
import datetime
import astropy
import astropy.time
import bokeh
import bokeh.io
import schedview
import erfa
import uranography.api
import rubin_sim
import rubin_scheduler
import schedview
import schedview.collect
import schedview.collect.visits
import schedview.plot
from IPython.display import HTML, display, Markdown, HTML

### Supress benign astropy warnings

In simulations, we use dates the astropy finds suspicious, or can't do "proper" sidereal time conversion for. Ignore.

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Don't even complain.
warnings.filterwarnings(
    "ignore",
    category=astropy.utils.exceptions.AstropyWarning,
    message="Tried to get polar motions for times after IERS data is valid. Defaulting to polar motion from the 50-yr mean for those. This may affect precision at the arcsec level. Please check your astropy.utils.iers.conf.iers_auto_url and point it to a newer version if necessary.",
)

# In simulations, we go far enough into the future that the erfa module finds it "dubious".
# Keep the complaints quiet.
warnings.filterwarnings(
    "ignore",
    category=erfa.ErfaWarning,
    message=r".*dubious year.*",
)

### Support the display of plots in the jupyter notebook

#### Support display of `bokeh` plots

In [ ]:
bokeh.io.output_notebook()

#### Support display of `matplotlib` plots

In [ ]:
%matplotlib inline

## Dealing with dayobs

`schedview` has a tool for dealing with the day of observation, as defined in SITCOMTM-032:

> A natural number representing the observation day (in timezone UTC-12:00) when
the takeImages command began executing. 


In instance of `schedview.DayObs` can be created either from any of several representations of a date:

In [ ]:
print(schedview.DayObs.from_date("20241123"))

In [ ]:
print(schedview.DayObs.from_date(20241123))

In [ ]:
print(schedview.DayObs.from_date("2023-11-23"))

In [ ]:
print(schedview.DayObs.from_date(60637, int_format="mjd"))

You can also create an instance from a time, in which case it will create an instance of `DayObs` for the date on which that time falls.

In [ ]:
print(schedview.DayObs.from_time("2024-11-24 08:00:00Z"))

In [ ]:
print(schedview.DayObs.from_time("2024-11-24 04:00:00Z"))

You can give it a time zone offset, following ISO-8601

In [ ]:
print(schedview.DayObs.from_time("2024-11-24 08:00:00-4"))

In [ ]:
print(schedview.DayObs.from_time("2024-11-24 04:00:00-4"))

You can also use an instance of `astropy.Time`,  or a floating point Modified Julian Date:

In [ ]:
t = astropy.time.Time("2024-11-24 08:00:00Z")
print(schedview.DayObs.from_time(t))

python's `datetime.datetime`:

In [ ]:
t = datetime.datetime(2024, 11, 24, 8, 0, 0, tzinfo=datetime.UTC)
print(schedview.DayObs.from_time(t))

or a floating point MJD (interperted in UTC, such that floating point MJDs early in the date correspend to the integer MJD of the day before):

In [ ]:
day_obs = schedview.DayObs.from_time(60638.2)
print(day_obs, day_obs.mjd)

An instance of `schedview.DayObs` can provide the date in a variety of formats:

In [ ]:
day_obs = schedview.DayObs.from_date("2023-11-23")

print("day_obs.yyyymmdd:", day_obs.yyyymmdd, type(day_obs.yyyymmdd))
print("day_obs.mjd:", day_obs.mjd, type(day_obs.mjd))
print("str(day_obs)", str(day_obs), type(str(day_obs)))
print("day_obs.date", day_obs.date, type(day_obs.date))
print("day_obs.start", day_obs.start, type(day_obs.start))
print("day_obs.end", day_obs.end, type(day_obs.end))

## Overall architecture of creating a plot

See the [architecture overview page](https://schedview.lsst.io/architecture.html) in the schedview documentation.

## Tables of visits

`schedview` can load tables of visits (represented as `pandas.DataFrame`s) either from the ConsDB or an opsim database. It can also use `maf` stackers (usually defined in `rubin_sim`) to supplement the columns that come directly from those sources.
There are three tools in `schedview` that return such tables:
- `schedview.collect.opsim.read_opsim`, which reads visits from an `opsim` database
- `schedview.collect.consdb.read_consdb`, which queries the consdb.
- `schedview.collect.visits.read_visits` is a more general interface which tries to automatically determine whether to query the `opsim` database for a baseline simulation or the consdb for a specific telescope, and calls either `schedview.collect.opsim.read_opsim` or `schedview.collect.consdb.read_consdb` to read visits *for one night*.

A number of other `schedview` functions expect visit tables in the form returned by these functions. 

### Reading visits from an `opsim` database

`schedview.collect.opsim.read_opsim` uses `rubin_sim.maf.get_sim_data` to load data visit, and returns it as a `pandas.DataFrame`.

In [ ]:
visits = schedview.collect.opsim.read_opsim(
    opsim_uri="/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs3.5/baseline/baseline_v3.5_10yrs.db",
    start_time=astropy.time.Time("2026-01-01 12:00:00Z"),
    end_time=astropy.time.Time("2026-01-02 12:00:00Z"),
)
visits.head()

Keywords not recognized by the `read_opsim` function itself are passed to `rubin_sim.maf.get_sim_data`.
This can be used, for example, to apply `maf` stackers:

In [ ]:
visits = schedview.collect.opsim.read_opsim(
    opsim_uri="/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs3.5/baseline/baseline_v3.5_10yrs.db",
    start_time=astropy.time.Time("2026-01-01 12:00:00Z"),
    end_time=astropy.time.Time("2026-01-02 12:00:00Z"),
    stackers=[rubin_sim.maf.HourAngleStacker()],
)
visits.loc[:, ["observationStartMJD", "fieldRA", "fieldDec", "filter", "HA"]].head()

A few `schedview` tools that take these tables of visits as arguments expect columns created using the stackers defined in `schedview.collect.visits.NIGHT_STACKERS`,
such that the `visits` table should be loaded thus:

In [ ]:
visits = schedview.collect.opsim.read_opsim(
    opsim_uri="/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs3.5/baseline/baseline_v3.5_10yrs.db",
    start_time=astropy.time.Time("2026-01-01 12:00:00Z"),
    end_time=astropy.time.Time("2026-01-02 12:00:00Z"),
    stackers=schedview.collect.visits.NIGHT_STACKERS,
)

### Reading visits from the ConsDB

Visits for a night or set of nights can also be queried from the ConsDB.
The columns returned include *both* the columns are they are natively named in the ConsDB, *and also* columns that match what `opsim` produces.
The result is redundant table: `fieldRA` and `s_ra` have the same data, for example.
But, this redundancy allows the table to be used where names according to what is in opsim are expected.

In [ ]:
visits = schedview.collect.consdb.read_consdb(
    "lsstcomcam",
    stackers=schedview.collect.visits.NIGHT_STACKERS,
    day_obs="2024-11-15",
    num_nights=1,
)
visits.head()

Note that `read_consdb` **does support** maf stackers.

### Generalize visit reading for a night

A common interface to collect data for one night from either a baseline simulation or the ConsDB, depending on provided parameters, simplifies the composition of notebooks that support the user specifying either:

To get visits from `lsstcomcam` from the ConsDB:

In [ ]:
visit_source = "lsstcomcam"
visits = schedview.collect.visits.read_visits(
    "2024-11-15", visit_source, stackers=schedview.collect.visits.NIGHT_STACKERS
)
visits.head()

To get visits from the 3.5 baseline:

In [ ]:
visit_source = "3.5"
visits = schedview.collect.visits.read_visits(
    "2025-11-15", visit_source, stackers=schedview.collect.visits.NIGHT_STACKERS
)
visits.head()

## Displaying visits

### Text tables

The simplest way to show the visits table is using the standard `pandas` display function, but it is impracticle to display more than a small fraction of the table with it.
`schedview` provides an additional tabular display for visits built on `bokeh`s `DataTable` tool.
While the `DataTable` tool works in most notebook environments and within Times Square, it does not work correctly an the notebook aspect of the RSP (apparently due to a conflict with firefly).

In [ ]:
%aimport schedview.examples.visittable
%psource schedview.examples.visittable

In [ ]:
p = schedview.examples.visittable.visit_table("3.5", "2025-11-15")
bokeh.io.show(p)

### Mapping the visits on the sky

In [ ]:
%aimport schedview.examples.visitmap
%psource schedview.examples.visitmap

In [ ]:
p = schedview.examples.visitmap.visit_map()
bokeh.io.show(p)

## Sun and moon positions

In [ ]:
%aimport schedview.examples.sunmoon
%psource schedview.examples.sunmoon

In [ ]:
schedview.examples.sunmoon.sun_moon_positions()

## Overhead summary table

In [ ]:
%aimport schedview.examples.overheadtable
%psource schedview.examples.overheadtable

In [ ]:
summary_table = schedview.examples.overheadtable.overhead_table()
display(HTML(summary_table))